In [1]:
import pandas as pd
from pathlib import Path

MAIN_PATH = Path(r"C:\Users\ag\alvaro\git\financial_planner")
BANK_DATA_PATH = MAIN_PATH / "bank_statements"
NORDEA_PATH = BANK_DATA_PATH / "nordea"
DANSKE_PATH = BANK_DATA_PATH / "danske"

In [2]:
from datetime import datetime, date
import plotly.express as px

import sys
PROJECT_ROOT = MAIN_PATH
sys.path.append(str(PROJECT_ROOT)) # Add project root to path
from src.data_loader import process_bank_data_folders, load_and_standardize_one_transaction_file
from src.categorizer import categorize_transactions_df, CATEGORY_RULES # Import your rules too
from src.utils import convert_currency_in_df

In [3]:
# Example paths—adjust to wherever you actually stored your CSVs:
nordea_csv = PROJECT_ROOT / "bank_statements/nordea/Grundkonto 3499420845 - 2024-01-21 19.29.04.csv"
danske_csv = PROJECT_ROOT / "bank_statements/danske/b_dashboard.csv"

# 2a. Load Nordea file
df_nordea = load_and_standardize_one_transaction_file(nordea_csv, bank_name="nordea")
print("Nordea shape:", df_nordea.shape)
display(df_nordea.head())

# 2b. Load Danske file
df_danske = load_and_standardize_one_transaction_file(danske_csv, bank_name="danske")
print("Danske shape:", df_danske.shape)
display(df_danske.head())

Nordea shape: (546, 7)


,Date,Description,Amount,Balance,Currency,Original_Bank,Status
0,NaT,IKEA COPENHAGEN DYBBOE,-1697.00,720112.30,DKK,Nordea,Reserved
1,2024-01-22,COOP365 SLUSEHOLMEN Den 19.01,-218.75,721809.30,DKK,Nordea,Booked
2,2024-01-19,COOP365 SLUSEHOLMEN Den 17.01,-83.87,722028.05,DKK,Nordea,Booked
3,2024-01-19,"Nordea pay, . COOP365 METROPOLEN Den 17.01",-18.00,722111.92,DKK,Nordea,Booked
4,2024-01-19,"Nordea pay køb, . IKEA.DK Den 17.01",-8536.00,722129.92,DKK,Nordea,Booked


Danske shape: (546, 7)


,Date,Description,Amount,Balance,Currency,Original_Bank,Status
0,NaT,IKEA COPENHAGEN DYBBOE,-1697.00,720112.30,DKK,Danske Bank,Pending/Unknown
1,2024-01-22,COOP365 SLUSEHOLMEN Den 19.01,-218.75,721809.30,DKK,Danske Bank,Booked
2,2024-01-19,COOP365 SLUSEHOLMEN Den 17.01,-83.87,722028.05,DKK,Danske Bank,Booked
3,2024-01-19,"Nordea pay, . COOP365 METROPOLEN Den 17.01",-18.00,722111.92,DKK,Danske Bank,Booked
4,2024-01-19,"Nordea pay køb, . IKEA.DK Den 17.01",-8536.00,722129.92,DKK,Danske Bank,Booked


In [4]:
df_nordea.head()

,Date,Description,Amount,Balance,Currency,Original_Bank,Status
0,NaT,IKEA COPENHAGEN DYBBOE,-1697.00,720112.30,DKK,Nordea,Reserved
1,2024-01-22,COOP365 SLUSEHOLMEN Den 19.01,-218.75,721809.30,DKK,Nordea,Booked
2,2024-01-19,COOP365 SLUSEHOLMEN Den 17.01,-83.87,722028.05,DKK,Nordea,Booked
3,2024-01-19,"Nordea pay, . COOP365 METROPOLEN Den 17.01",-18.00,722111.92,DKK,Nordea,Booked
4,2024-01-19,"Nordea pay køb, . IKEA.DK Den 17.01",-8536.00,722129.92,DKK,Nordea,Booked


In [5]:
file_list = [
    (PROJECT_ROOT / "bank_statements/nordea/Grundkonto 3499420845 - 2024-01-21 19.29.04.csv", "nordea"),
    (PROJECT_ROOT / "bank_statements/nordea/Konto 3499420845 - 2025-06-05 10.14.07.csv", "nordea2"),
    #(PROJECT_ROOT / "bank_statements/danske/b_dashboard.csv", "danske"),

]

all_dfs = []
for path, bank in file_list:
    df = load_and_standardize_one_transaction_file(path, bank_name=bank)
    if not df.empty:
        all_dfs.append(df)

# concatenate
combined = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()
print("Combined before dedupe:", combined.shape)
df_all = combined.copy()
df_all.head()

Combined before dedupe: (1212, 7)


,Date,Description,Amount,Balance,Currency,Original_Bank,Status
0,NaT,IKEA COPENHAGEN DYBBOE,-1697.00,720112.30,DKK,Nordea,Reserved
1,2024-01-22,COOP365 SLUSEHOLMEN Den 19.01,-218.75,721809.30,DKK,Nordea,Booked
2,2024-01-19,COOP365 SLUSEHOLMEN Den 17.01,-83.87,722028.05,DKK,Nordea,Booked
3,2024-01-19,"Nordea pay, . COOP365 METROPOLEN Den 17.01",-18.00,722111.92,DKK,Nordea,Booked
4,2024-01-19,"Nordea pay køb, . IKEA.DK Den 17.01",-8536.00,722129.92,DKK,Nordea,Booked


In [6]:
df_all = categorize_transactions_df(df_all, CATEGORY_RULES)
df_all.head()

,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category
0,NaT,IKEA COPENHAGEN DYBBOE,-1697.00,720112.30,DKK,Nordea,Reserved,Household
1,2024-01-22,COOP365 SLUSEHOLMEN Den 19.01,-218.75,721809.30,DKK,Nordea,Booked,Groceries
2,2024-01-19,COOP365 SLUSEHOLMEN Den 17.01,-83.87,722028.05,DKK,Nordea,Booked,Groceries
3,2024-01-19,"Nordea pay, . COOP365 METROPOLEN Den 17.01",-18.00,722111.92,DKK,Nordea,Booked,Groceries
4,2024-01-19,"Nordea pay køb, . IKEA.DK Den 17.01",-8536.00,722129.92,DKK,Nordea,Booked,Household


In [7]:
df_all[df_all['Category']=='Uncategorized'].sort_values('Amount').head(10)

,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category
29,2023-12-28,MobilePay Niklas Thr,-5500.00,735843.96,DKK,Nordea,Booked,Uncategorized
1188,2023-12-28,MobilePay Niklas Thr,-5500.00,735843.96,DKK,Nordea2,Booked,Uncategorized
1049,2024-04-15,MobilePay Beatriz Ec,-5000.00,795492.59,DKK,Nordea2,Booked,Uncategorized
986,2024-06-06,"EUR 640,56 IBEXPRESS06023073144",-4802.19,823653.06,DKK,Nordea2,Booked,Uncategorized
483,2023-01-23,MobilePay beatriz ec,-3900.00,637887.60,DKK,Nordea,Booked,Uncategorized
340,2023-05-11,"EUR 357,30 SOHO BOUTIQUE CADIZ",-2691.28,694006.56,DKK,Nordea,Booked,Uncategorized
311,2023-06-06,"Nordea pay køb, . MOBILEPAY Den 03.06",-2500.00,709363.44,DKK,Nordea,Booked,Uncategorized
821,2024-10-02,"EUR 299,81 GRUPOGALDANA",-2246.95,879772.45,DKK,Nordea2,Booked,Uncategorized
1024,2024-05-03,"EUR 277,40 IBEXPRESS06023071989",-2079.62,807946.77,DKK,Nordea2,Booked,Uncategorized
125,2023-10-27,"EUR 266,06 IBEXPRESS06023064655",-1996.24,735622.19,DKK,Nordea,Booked,Uncategorized


In [7]:
start_date = '2023-01-01'
end_date = '2024-01-01'

df_all[(df_all['Date']>=start_date)&
       (df_all['Date']<end_date)]\
       .sort_values('Date').head()

,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category
509,2023-01-02,"Nordea pay køb, . FAKTA METROPOLEN Den 29.12",-49.00,645387.18,DKK,Nordea,Booked,Groceries
503,2023-01-02,"Nordea pay, . COOP 365 KINGOSVEJ Den 31.12",-210.00,644410.78,DKK,Nordea,Booked,Uncategorized
504,2023-01-02,Bs betaling FITNESS WORLD A/S,-258.00,644620.78,DKK,Nordea,Booked,Sports
508,2023-01-02,"REMA 1000 KBH., BRYGDen 29.12",-34.95,645352.23,DKK,Nordea,Booked,Uncategorized
506,2023-01-02,"Nordea pay køb, . MOBILEPAY Den 30.12",-269.00,645057.23,DKK,Nordea,Booked,Uncategorized


In [8]:
df_all['Category']

0           Household
1           Groceries
2           Groceries
3           Groceries
4           Household
            ...      
541        Dining Out
542         Utilities
543         Groceries
544    Internet/Phone
545            Sports
Name: Category, Length: 546, dtype: object

In [9]:
# Check that ‘Date’, ‘Description’, ‘Amount’, ‘Balance’, ‘Currency’, ‘Original_Bank’, ‘Status’ are present:
print("Columns:", df_all.columns.tolist())

# Check for any rows with missing Date or Amount
missing_date = df_all[df_all["Date"].isna()]
print("Rows with invalid/missing Date:", len(missing_date))
display(missing_date.head())

missing_amount = df_all[df_all["Amount"].isna()]
print("Rows with invalid/missing Amount:", len(missing_amount))
display(missing_amount.head())

Columns: ['Date', 'Description', 'Amount', 'Balance', 'Currency', 'Original_Bank', 'Status', 'Category']
Rows with invalid/missing Date: 1


,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category
0,NaT,IKEA COPENHAGEN DYBBOE,-1697.0,720112.3,DKK,Nordea,Reserved,Household


Rows with invalid/missing Amount: 0


,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category


In [10]:
from src.utils import convert_currency_in_df

df_eur = convert_currency_in_df(df_all, target_currency="EUR")
print("Preview EUR conversion:")
display(df_eur.head())

Preview EUR conversion:


,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category
0,NaT,IKEA COPENHAGEN DYBBOE,-227.479893,96529.798928,EUR,Nordea,Reserved,Household
1,2024-01-22,COOP365 SLUSEHOLMEN Den 19.01,-29.323056,96757.278820,EUR,Nordea,Booked,Groceries
2,2024-01-19,COOP365 SLUSEHOLMEN Den 17.01,-11.242627,96786.601877,EUR,Nordea,Booked,Groceries
3,2024-01-19,"Nordea pay, . COOP365 METROPOLEN Den 17.01",-2.412869,96797.844504,EUR,Nordea,Booked,Groceries
4,2024-01-19,"Nordea pay køb, . IKEA.DK Den 17.01",-1144.235925,96800.257373,EUR,Nordea,Booked,Household


In [11]:
# 7a. Date range
df_all["Date"] = pd.to_datetime(df_all["Date"], errors="coerce")
min_date, max_date = df_all["Date"].min().date(), df_all["Date"].max().date()
print("Date range:", min_date, "—", max_date)

# Example: only keep Jan–May 2025
from datetime import date
start_date = date(2023, 1, 1)
end_date   = date(2025, 5, 31)

mask = (df_all["Date"].dt.date >= start_date) & (df_all["Date"].dt.date <= end_date)
df_filtered = df_all.loc[mask].copy()
print("After date filter:", df_filtered.shape)

# 7b. Category check (if you have categorization enabled)
if "Category" not in df_filtered.columns or df_filtered["Category"].isna().all():
    from src.categorizer import categorize_transactions_df, CATEGORY_RULES
    df_filtered = categorize_transactions_df(df_filtered.copy(), CATEGORY_RULES)

print("Unique categories:", df_filtered["Category"].unique())


Date range: 2022-12-01 — 2024-01-22
After date filter: (509, 8)
Unique categories: ['Groceries' 'Household' 'Shopping' 'Uncategorized' 'Transport'
 'Utilities' 'Sports' 'Rent Flat' 'Bank Interest' 'Financial/Fees'
 'Internet/Phone' 'Salary' 'Revolut transfers' 'Cash Withdrawal'
 'Dining Out' 'Healthcare' 'Transfers']


In [12]:
# 8a. Transactions where Status != "Booked"
print("Non-booked rows:")
display(df_all[df_all["Status"] != "Booked"].head())

# 8b. Large positive or negative amounts
big_tx = df_all[ df_all["Amount"].abs() > 10_000 ]
print("Transactions > |10,000| DKK:")
display(big_tx.sort_values("Amount", ascending=False).head())

Non-booked rows:


,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category
0,NaT,IKEA COPENHAGEN DYBBOE,-1697.0,720112.3,DKK,Nordea,Reserved,Household


Transactions > |10,000| DKK:


,Date,Description,Amount,Balance,Currency,Original_Bank,Status,Category
209,2023-08-31,LØNOVERFØRSEL,37905.85,755131.69,DKK,Nordea,Booked,Salary
28,2023-12-29,LØNOVERFØRSEL,36986.33,772830.29,DKK,Nordea,Booked,Salary
55,2023-11-30,LØNOVERFØRSEL,36986.33,760871.05,DKK,Nordea,Booked,Salary
117,2023-10-31,LØNOVERFØRSEL,36986.33,757596.70,DKK,Nordea,Booked,Salary
175,2023-09-29,LØNOVERFØRSEL,36986.33,785998.48,DKK,Nordea,Booked,Salary


# Graphs

In [13]:
start_date = '2022-12-01'
end_date = '2024-01-01'

filtered_df = df_all[(df_all['Date']>=start_date)&
       (df_all['Date']<end_date)]\
       .sort_values('Date')


expenses_df = filtered_df[filtered_df['Amount'] < 0].copy()
expenses_df['Absolute_Amount'] = expenses_df['Amount'].abs()
category_spending = expenses_df.groupby('Category')['Absolute_Amount'].sum().sort_values(ascending=False)
category_spending


Category
Rent Flat            137911.00
Uncategorized        113587.47
Transport             15330.93
Groceries             12261.47
Revolut transfers     12198.99
Shopping               8491.74
Utilities              3772.88
Internet/Phone         3637.00
Sports                 3334.00
Dining Out             1695.32
Household               962.00
Cash Withdrawal         878.53
Financial/Fees          810.00
Healthcare              152.75
Name: Absolute_Amount, dtype: float64

In [14]:
monthly_df = filtered_df.copy()
monthly_df['Date'] = pd.to_datetime(monthly_df['Date'])
monthly_df.set_index('Date', inplace=True)

monthly_summary = monthly_df.resample('M')['Amount'].agg(
    Income=lambda x: x[x > 0].sum(),
    Expenses=lambda x: x[x < 0].sum() # Expenses are negative, sum will be negative
).reset_index()
monthly_summary['Expenses'] = monthly_summary['Expenses'].abs() # Make expenses positive for plotting alongside income
monthly_summary


C:\Users\ag\AppData\Local\Temp\ipykernel_9764\2603062748.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_summary = monthly_df.resample('M')['Amount'].agg(


,Date,Income,Expenses
0,2022-12-31,33739.33,17623.29
1,2023-01-31,34135.23,21250.53
2,2023-02-28,33229.33,21815.96
3,2023-03-31,34179.33,18843.76
4,2023-04-30,34305.33,15812.25
5,2023-05-31,36107.23,27246.28
6,2023-06-30,32581.33,19250.88
7,2023-07-31,32838.00,18627.72
8,2023-08-31,38506.50,23628.39
9,2023-09-30,36986.33,6119.54


In [15]:
if not monthly_df.empty and 'Amount' in monthly_df.columns and pd.api.types.is_numeric_dtype(monthly_df['Amount']):
    monthly_net_savings = monthly_df.resample('ME')['Amount'].sum().reset_index() # Ensure 'Date' becomes a column
    monthly_net_savings.rename(columns={'Amount': 'Net Savings'}, inplace=True) # Rename summed column
    
monthly_net_savings

,Date,Net Savings
0,2022-12-31,16116.04
1,2023-01-31,12884.70
2,2023-02-28,11413.37
3,2023-03-31,15335.57
4,2023-04-30,18493.08
5,2023-05-31,8860.95
6,2023-06-30,13330.45
7,2023-07-31,14210.28
8,2023-08-31,14878.11
9,2023-09-30,30866.79


In [16]:
if not monthly_net_savings.empty:
    fig_net_savings = px.bar(
        monthly_net_savings, 
        x='Date',  # Make sure 'Date' is the x-axis
        y='Net Savings', # Make sure 'Net Savings' is the y-axis
        title="Monthly Net Savings"
    )
    fig_net_savings.update_layout(yaxis_title=f"Net Savings ()")
    fig_net_savings.add_hline(y=0, line_dash="dash", line_color="red")
fig_net_savings

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Date=%{x}<br>Net Savings=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2022-12-31T00:00:00.000000000', '2023-01-31T00:00:00.000000000',
                          '2023-02-28T00:00:00.000000000', '2023-03-31T00:00:00.000000000',
                          '2023-04-30T00:00:00.000000000', '2023-05-31T00:00:00.000000000',
                          '2023-06-30T00:00:00.000000000', '2023-07-31T00:00:00.000000000',
                          '2023-08-31T00:00:00.000000000', '2023-09-30T00:00:00.000000000',
                          '2023-10-31T00:00:00.000000000', '2023-11-30T00:00:00.000000000',
                          '2023-12-31T00:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('7FG4HgV6z0CamZmZWSrJQMT1KFyvSs' ... '0KZ5TcwEjhehQuKrBAPgrXo3DPx0A='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'shapes': [{'line': {'color': 'red', 'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 0,
                           'y1': 0,
                           'yref': 'y'}],
               'template': '...',
               'title': {'text': 'Monthly Net Savings'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Net Savings ()'}}}
})